# Intro to machine learning - Scikit-Learn

**Before this notebook, you should look at [Intro_to_pandas.ipynb](Intro_to_pandas.ipynb).**

In this notebook, we'll explore the Scikit Learn package for simple machine learning tasks using geoscience data examples. After this day, students will have a good overview of how to look at large datasets and solve problems with state-of-the-art machine learning tools.

- Machine learning concepts
- What is it that you’re trying to solve? How can machine learning help?
- What's the difference between supervised and unsupervised methods?
- What's the difference between classification and regression?


<img src="../images/ML_loop.png"></img>

### The machine learning iterative loop
- Data — Getting the data. How to load it and put it in an `array` and/or `DataFrame`
- Processing — data exploration, inspection, cleaning, and feature engineering.
- Model – What is a model? Training a Scikit-Learn model.
- Results – assessing quality and performance metrics (accuracy, recall, F1, confusion matrices)
- Repeat – What can we do to improve performance?

<img src="../images/machine_learning_primer.png"></img>

# Import libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load the data

In [ ]:
import pandas as pd
df = pd.read_csv('../data/training_DataFrame_processed.csv')

---
# Scikit-learn classifiers

Let's create a model that classifies between those three classes.

## Set up the task

In [ ]:
df.head()

In [ ]:
# Make X and y
X = df[['GR','RHOB','PE','ILD_log10']].values
y = df['s_Facies'].values

Some methods expect the data to be normalized. It's likely a good idea to normalize it no matter which method you try.

`scikit-learn` has lots of scalers. The `StandardScaler` removes the mean and scales the data to unit variance.

Let's take a quick look at the data before scaling:

In [ ]:
plt.scatter(X[:,0], X[:,1], c=df['Facies'].values)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
plt.scatter(X[:,0], X[:,1], c=df['Facies'].values)

## Split the data

We must split the data into a training set, a validation set, and a test set. **This is a key step in the process.**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.4, random_state=42)

We will also split this second set (with the underscores) into two parts: one to **validate** against while training the model and selecting hyterparameters (sometimes also called the **dev** set), and one to assess the likely real-world performance of the trained model.

Note that you should only predict on the **test** set once, at the end of model selection and tuning.

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5, random_state=42)

How do we feel about this? We've drawn all our records randomly from our data.

### The IID assumption

Our data records are not strictly independent and identically distributed. So splitting like this is not a great idea for these data. We should split by well instead.

In [ ]:
df['Well Name'].unique()

In [ ]:
train_wells = ['CHURCHMAN BIBLE', 'CROSS H CATTLE', 'LUKE G U', 'NOLAN', 'SHANKLIN', 'Recruit F9']
features = ['GR','RHOB','PE','ILD_log10']

X_train = df.loc[df['Well Name'].isin(train_wells), features].values
y_train = df.loc[df['Well Name'].isin(train_wells), 's_Facies'].values

In [ ]:
X_train.shape, y_train.shape

In [ ]:
val_wells = ['NEWBY', 'SHRIMPLIN']

X_ = df.loc[df['Well Name'].isin(val_wells), features].values
y_ = df.loc[df['Well Name'].isin(val_wells), 's_Facies'].values

In [ ]:
X_.shape, y_.shape

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5, random_state=42)

## A simple model: _k_-NN

A fairly common method for classifying data is the _k-nearest neighbours algorithm_. The label of the object in question is determined by the neighbouring data points in the feature space used. Its most important parameter, _k_ , called `n_neighbors`, is the number of neighbors you include to make a membership decision.

First we import, then instantiate, the classifier:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()

### `fit` (train)

The next block is all you need to train a classifier model!

In [ ]:
clf.fit(X_train, y_train)

### `predict`

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(set(y_train)))
print(le.classes_)
print(le.transform(le.classes_))

y_val_int = le.transform(y_val) 
y_pred_int = le.transform(y_pred)

In [ ]:
from matplotlib.colors import ListedColormap

fig, axs = plt.subplots(ncols=3, figsize=(5, 10), sharey=True)

colours = ['#2E86C1','#F4D03F', '#1B4F72']
cmap_facies = ListedColormap(colours, 'indexed')

ax = axs[0]
im = ax.imshow(y_val_int[:100].reshape(-1, 1), aspect=0.05, cmap=cmap_facies)
ax.set_title('Actual')

ax = axs[1]
im = ax.imshow(y_pred_int[:100].reshape(-1, 1), aspect=0.05, cmap=cmap_facies)
ax.set_title('Predicted')

ax = axs[2]
ax.axis('off')
cbar = plt.colorbar(im, ax=ax)
cbar.set_ticks([0, 1, 2])
cbar.set_ticklabels(le.classes_)

plt.show()

### `score`

Looking at the results is great, but we need to get quantitative if we want to make sure that the model we trained is _good_ and produces reasonable results. The most basic test is to look at how many good predictions we would make if we predict on our **validation** data.

In [ ]:
score = clf.score(X_val, y_val)
print(f"The accuracy is {score*100:.1f}%")

This is the same as explicitly calling `sklearn.metrics.accuracy_score()` on the validation labels and the prediction from the validation data.

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_val, y_pred))

The _accuracy_ is just one of the _metrics_ we can use to check the quality of the predictions. There are a large number of different metrics and depending on your data and problem you may need to find the one that adjusts better to your needs.

In general, _accuracy_ can be misleading, especially in datasets with unbalanced classes. A more robust metric is the `F1` metric. It combines the `precision` score and `recall`:

$$ \mathrm{F1} = \frac{2}{\frac{1}{\mathrm{precision}}+ \frac{1}{\mathrm{recall}}} $$

Scikit-learn gives a nice summary of these three metrics using `classification_report`.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, clf.predict(X_val), digits=3))

In [ ]:
from IPython.display import Image
Image('../images/classification_metric_definitions.png', width=400) 

<div class="alert alert-success">
<h3>Exercise</h3>

- what is the **accuracy** of this classifier?
- what is the **precision** for dolomite and sand? 
- what is the **recall** for dolomite and sand? 
- what is the **F1 score** for dolomite and sand? 
- Discuss with a partner the situations where accuracy or F1 score can be misleading.

Note that there are 31 dolomite points and 55 sandstone points in the validation data.
</div>

In [ ]:
Image('../images/2d_2class_classifier.png') 

In [ ]:
Image('../images/dolomite_and_sandstone_worksheet.png') 

In [ ]:
from IPython.display import Image
Image('../images/clf_report.png') 

## Improving the performance

Depending on you requirements, this results might be good enough to deploy this model and use it on a "Machine Learning Pipeline" product but it is often not the best model you can get. Each method has a set of parameters (also known as _hyperparameters_) that can be tweaked to tune the training.

For the `KNeighborsClassifier` there are a few of these parameters:

In [ ]:
KNeighborsClassifier()

For this particular method, the most important parameter to adjust is `n_neighbors` (it's the `k` in the `KNeighborsClassifier`!). Unfortunately, there's no rule that tells you what's the optimal value of `k`. To overcome this we can train many models with different values of `k` and compare the results of classifications applied to the _Validation_ data.

In [ ]:
k = np.arange(1, 60, 2) # Generated array of values of k to try

Loop over each value in `nns` and store the `F1 Score`

In [ ]:
from sklearn.metrics import f1_score

vals, trns = [], []
pvals, ptrns = [], []

for ki in k:
    clf = KNeighborsClassifier(n_neighbors=ki)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_val)
    vals.append(f1_score(y_val, y_pred, average='weighted'))
    y_ptrn = clf.predict(X_train)
    trns.append(f1_score(y_train, y_ptrn, average='weighted'))


What value of `k` gives us the best result?

In [ ]:
plt.plot(k, vals, lw=3.0, label="Validation F1 score")
plt.plot(k, trns, lw=3.0, label="Training F1 score")
_ = plt.xlabel('n_neighbors')
_ = plt.ylabel('clf.score')
_ = plt.legend()

<div class="alert alert-success">
<h3>Exercise</h3>

- Create a new `KNeighborsClassifier` classifier where you specify the optimal number of neighbours
- Write a new classification report for the new classifier
</div>

## Using the classifier to make predictions

Say you've collected / acquired some new data. How would you use your classifier on it to make predictions? 

- Load it

  `new_data = pd.read_csv('..some_new_data_you_have_collected.csv')`
  

- Extract the relavant features and cast it as a 2-d array

  `new_X = new_data[['GR','RHOB','PE','ILD_log10']].values`


- Apply the same tranformation equations to each feature

  `new_X = scalar.fit_transform(new_X)`


- Pass it into the classifier's predict method

  `y_pred = clf.predict(new_X)`
  

## More methods to train models!

Let's pick 3 different classifiers to train different models and then compare how well they perform

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
classifiers = {
    "Linear SVM": SVC(),
    "Random forest": RandomForestClassifier(),
    "Neural network": MLPClassifier(),
}

In [ ]:
classifiers

Let's iterate over these classifiers and print common metrics to evaluate the performance of each model using the testing dataset we defined before

In [ ]:
# iterate over classifiers
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    score = clf.score(X_val, y_val)
    print("{:12} {}".format(name,"-"*15))
    print(classification_report(y_val, clf.predict(X_val), digits=3))

<div class="alert alert-success">
<h3>Exercise</h3>

- Try other methods available in the scikit-learn library. See the list [here](http://scikit-learn.org/stable/supervised_learning.html)
</div>

### Choosing the right estimator

Often the hardest part of solving a machine learning problem can be finding the right estimator for the job.

Different estimators are better suited for different types of data and different problems.

#### For a classifier comparison check the source code [here](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)

<img src="../images/ML_classifier_comparison_sklearn.png"></img>

Let's try visualizing the decision boundary for our problem.

# Parameter selection

Many of the models can be improved (or worsened) by changing the parameters that internally make the method work. It's always a good idea to check the documentation of each model (e.g. `RandomForestClassifier` [docs](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). This process is usually called _hyperparameter tuning_.

Scikit-learn offers a simple way to test different parameters for each model through a function called `GridSearchCV`

The grid search does cross-validation, instead of a validation set. So let's add our training and validation data together, so we get to use all of it.

In [ ]:
X_all = np.vstack([X_train, X_val])
y_all = np.hstack([y_train, y_val])

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'min_samples_leaf':np.arange(1, 26, 1),
              'max_depth':np.arange(1, 16)}

rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, parameters, iid=False, cv=6, n_jobs=4, verbose=1)
clf.fit(X_all, y_all)

How does the parameter space look like with respect to the score of the classifier?

In [ ]:
scores = clf.cv_results_['mean_test_score']
max_depths = clf.cv_results_["param_max_depth"].data.astype(int)
min_samples_leaf = clf.cv_results_["param_min_samples_leaf"].data.astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
im = ax.imshow(scores.reshape((15, 25)),
               origin='lower',
               extent=[0.5, 25.5, 0.5, 15.5],
               interpolation='none'
              )
ax.set_ylabel('max_depth')
ax.set_xlabel('min_samples_leaf')
cb = plt.colorbar(im, shrink=0.75)

`clf` can now tell us the best parameters to use with our `RandomForestClassifier`

In [ ]:
clf.best_params_

In [ ]:
clf.best_estimator_

The nice thing about `scikit-learn`'s methods is that they're all consistent and behave in the same way. Notice how`GridSearchCV` was `.fit()`. That means that we can use it to `.predict()` and it will automatically use the best set of parameters!

In [ ]:
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred, digits=3))

Now might be a good time to check our **test** set — the one we reserved at the start. How might our model do on this dataset?

In [ ]:
y_check = clf.predict(X_test)
print(classification_report(y_test, y_check, digits=3))

We get the same performance as on the validation set! This is a good result, and gives us comfort that our model is going to do a reasonable job.

## Confusion matrix

It's also helpful to summarize the prediction tests using a [Confusion Matrix](https://en.wikipedia.org/wiki/Confusion_matrix). Scikit-learn has a function for that!

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, y_pred)

But as you can see, it's not very clear... What does each row/column represent? We can help a bit:

In [ ]:
from collections import Counter

Counter(y_val)

In [ ]:
selected = np.unique(y_val)

In [ ]:
# itertoools is a standard library for all kinds of handy iterator manipulation
import itertools

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_val, y_pred)

title = 'Confusion matrix'
cmap = plt.cm.Reds

# Plot non-normalized confusion matrix.
plt.imshow(cnf_matrix, interpolation='nearest', cmap=cmap)
plt.title(title)
plt.colorbar()
tick_marks = np.arange(len(selected))
plt.xticks(tick_marks, selected, rotation=45)
plt.yticks(tick_marks, selected)

# Print the support numbers inside the plot.
thresh = cnf_matrix.max() / 2.
for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
    plt.text(j, i, format(cnf_matrix[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cnf_matrix[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

## Model persistence

Often, we'd like to save the trained model, to go and apply it in some other application, or to share with someone else. The easiest way to save most models is as a Python 'pickle' object:

In [ ]:
from sklearn.externals import joblib
joblib.dump(clf, 'facies_model.pkl')

How do you load a saved model?

In [ ]:
clf = joblib.load('facies_model.pkl')

---
# Where to go next?

- More data!
- [XGBoost](https://xgboost.readthedocs.io/en/latest/)
- [LightGBM](https://github.com/Microsoft/LightGBM)
- If you want to get started on Neural Networks, [Keras](https://keras.io/) provides a scikit-learn type of experience

### Paper with classifier comparison ([link](https://arxiv.org/abs/1708.05070))

<img src="../data/model_performance.jpg"></img>

## The Data Science Hierarchy of Needs ([article](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007))

<img src="../images/the_ai_hierarchy_of_needs.png"></img>

## Nuance

- Data normalization doesn't magically fix problems with data scaling. If you're lumping a bunch of well data together and the GR, say, has different ranges in each well, then the scaled data will also have this problem. So you still need to calibrate or normalize your data to ensure it's internally consistent. This is your reponsibility; `scikit-learn` scalers don't do this for you.

- Class imbalance is also your problem. You need to make sure you have good representation from all classes.


<hr />

<p style="color:gray">©2017 Agile Geoscience. Licensed CC-BY.</p>